In [6]:
import os
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from dataset import load_core_set_data, load_pca_data
from simulation.simulator import FDTDSimulator
from simulation.student import LSTMPredictor

# 加载数据集

In [7]:
class FDTDDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.FloatTensor(data)
        self.labels = torch.FloatTensor(labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


train_data, train_labels, test_data, test_labels = load_core_set_data()

train_dataset = FDTDDataset(train_data, train_labels)
test_dataset = FDTDDataset(test_data, test_labels)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


In [8]:
radius_matrix = torch.rand(10, 10) * 10

print(radius_matrix)

tensor([[2.9284, 8.1854, 5.4482, 2.8033, 2.4474, 7.3576, 6.0576, 1.8627, 6.5422,
         2.0916],
        [8.4445, 5.2005, 6.1428, 3.4576, 0.2301, 6.9667, 9.7425, 1.8224, 6.9885,
         5.3009],
        [4.6831, 2.8447, 0.4183, 1.6436, 5.7233, 3.0953, 4.0740, 2.4000, 3.7076,
         1.9033],
        [6.1650, 1.7961, 2.2919, 5.4673, 9.0467, 9.5927, 4.3925, 3.8689, 2.7298,
         1.8522],
        [9.7205, 8.5215, 1.5024, 0.5147, 5.2148, 7.0771, 2.0613, 0.6072, 6.9567,
         7.8390],
        [6.7288, 4.2543, 2.0420, 1.5554, 2.3349, 8.7234, 0.2983, 6.5317, 5.8932,
         0.4964],
        [6.1732, 5.0258, 9.5539, 2.5665, 5.5947, 4.9657, 0.9088, 4.0744, 2.2189,
         3.8342],
        [7.7659, 7.9777, 4.7939, 0.5960, 4.6185, 1.0744, 2.7790, 5.0916, 0.1865,
         6.9825],
        [9.4482, 2.5026, 0.9727, 3.3313, 0.9315, 8.6481, 8.8279, 4.8965, 1.1942,
         0.7872],
        [1.0959, 2.6780, 2.4197, 2.3690, 3.5365, 7.9677, 5.3850, 3.4023, 9.3465,
         5.4040]])


In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = LSTMPredictor(
    input_size=10,
    hidden_size=128,
    output_size=10,
    num_layers=2,
    dropout=0.1,
    device=device
)
simulator = FDTDSimulator(
    radius_matrix=radius_matrix
)

num_epochs = 100
learning_rate = 1e-3

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

In [10]:
model.train()

total_steps = num_epochs * len(train_loader)
progress_bar = tqdm(total=total_steps, desc="Training Progress")

for epoch in range(num_epochs):
    epoch_loss = 0
    
    for inputs, _ in train_loader:
        inputs = inputs.to(device)
        optimizer.zero_grad()

        with torch.no_grad():
            target_seq = simulator(inputs).detach()

        with torch.enable_grad():
            _, pred_seq = model.get_sequence_output(inputs)

            loss = criterion(pred_seq, target_seq)
            loss.backward()
            optimizer.step()
        
        epoch_loss += loss.item()
        # 更新进度条
        progress_bar.update(1)
        progress_bar.set_description(f"Loss: {loss.item():.6f}")
    
    avg_loss = epoch_loss / len(train_loader)
    tqdm.write(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.6f}")

progress_bar.close()

Loss: 0.058759:   0%|          | 3/600000 [00:34<1942:47:26, 11.66s/it]


KeyboardInterrupt: 

In [ ]:
save_dir = "data/model/lstm_model.pth"
os.makedirs(os.path.dirname(save_dir), exist_ok=True)

torch.save({
    "model_state_dict": model.state_dict(),
    "radius_matrix": radius_matrix
}, save_dir)